# Sharpening Notebook
sharpening.bit location: \xilinx\pynq\overlays\base

In this notebook, a gaussian blur and laplacian will be applied to HDMI input stream and added back to the orignial input. Those input sources and applied filters will then be displayed on a HDMI output.

### Load the overlay

In [2]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.video import *

base = Overlay("/home/xilinx/pynq/overlays/base/sharpening.bit")
base.download()

### Initialize HDMI

In [3]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

 
hdmi_in.configure(PIXEL_RGB)
hdmi_in.start()
 
hdmi_out.configure(hdmi_in.mode)
hdmi_out.start()

In [4]:
hdmi_in.tie(hdmi_out)

### Initialize Slider

In [5]:
from functools import wraps
def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f



In [6]:
sharpining_ip = MMIO(0x43C00000,0x10000)

from ipywidgets import IntSlider, VBox, HTML
IntensityLaplacian=IntSlider()

IntensityLaplacian.max = 300
IntensityLaplacian.step = 50

@yield_for_change(IntensityLaplacian, 'value')
def f():
    for i in range(50000):
        x = yield
        sharpining_ip.write(0x10,x)
        
f()

### Slider:

The slider changes the intensity of the laplacian frame in percentage (100 = 100% intensity).

In [7]:
IntensityLaplacian

IntSlider(value=0, max=300, step=50)

In [8]:
hdmi_out.close()
hdmi_in.close()